In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import datetime
from sklearn.model_selection import train_test_split

import torch
import re
import os
import shutil
import glob
import zipfile
import cv2

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print('device :', device)
print('Current :', torch.cuda.current_device())
print('Count :', torch.cuda.device_count())

device : cuda
Current : 0
Count : 1


## 1. 데이터 폴더 생성 (1회만 실행)

In [5]:
# female 엑셀파일에 id가 겹치는게 있음 -> male 엑셀파일도 있을 수 있음
# 남여 나눠서 압축 1개 풀고, 엑셀 1개 concat 하는 식으로 수행
# 이미지 이름은 1부터 rename 하였음

path= '골밀도 데이터/BoneAge/'
data_path_F = '골밀도 데이터/data_F/'
data_path_M = '골밀도 데이터/data_M/'

# xlsx파일과 zip파일 경로 추출
xls_path = glob.glob(path + "*.xlsx")
zip_path = glob.glob(path + "*.zip")

In [6]:
# 남여 zip 파일 분류
zip_path_F = [f_path for f_path in zip_path if f_path.split('/')[-1][3]=='F']
zip_path_M = [f_path for f_path in zip_path if f_path.split('/')[-1][3]=='M']

# 남여 xlsx 파일 분류
xlsx_path_F = [f_xls for f_xls in xls_path if f_xls.split('/')[-1][3]=='F']
xlsx_path_M = [m_xls for m_xls in xls_path if m_xls.split('/')[-1][3]=='M']

## 분류된 파일을 보면 연령대 순서가 동일함을 알 수 있음
# print(zip_path_M)
# print(xlsx_path_M)

# 이미지 이름을 1부터 rename
i = 1
df_f = pd.DataFrame() # female 엑셀파일 모아둘 데이터프레임 생성

# zip 함수로 female zip파일, 엑셀파일 세트로 하나씩 추출
for zip_file, xlsx_file in zip(zip_path_F, xlsx_path_F):
    zipfile.ZipFile(zip_file).extractall(data_path_F)
    xlsx_f = pd.read_excel(xlsx_file, index_col = 0)
    df_f = pd.concat([df_f, xlsx_f], axis = 0)
    
    for filename in os.listdir(data_path_F):
        # 01_F 식으로 된 파일명만 바꾸기
        if filename.split('.')[0][-1] == 'F':
            new_filename = str(i) + '.jpg'
            
            old_path = os.path.join(data_path_F, filename)
            new_path = os.path.join(data_path_F, new_filename)
            
            os.rename(old_path, new_path)
            i+= 1
        else:
            continue

df_f = df_f.reset_index(drop = True)

In [7]:
# 이중 폴더 꺼내는 것 제외하면 female 작업과 같음
for zip_file in zip_path_M:
    zipfile.ZipFile(zip_file).extractall(data_path_M)

df_m = pd.DataFrame()
folder_path = glob.glob(data_path_M + "BA*")
for in_path, xlsx_file in zip(folder_path, xlsx_path_M):
    xlsx_m = pd.read_excel(xlsx_file, index_col=0)
    df_m = pd.concat([df_m, xlsx_m], axis = 0)
    
    temp = os.listdir(in_path)
    for tmp in temp:
        os.rename(in_path + '/' + tmp, data_path_M + str(i) + '.jpg')
        i+= 1
    
    os.rmdir(in_path)
    
df_m = df_m.reset_index(drop = True)

In [8]:
# 두 데이터를 합치기 위한 data 폴더 생성
merged_folder = '골밀도 데이터/data/'
os.makedirs(merged_folder, exist_ok=True)

# data_F의 이미지를 data 폴더로 이동
for filename in os.listdir(data_path_F):
    source_path = os.path.join(data_path_F, filename)
    destination_path = os.path.join(merged_folder, filename)
    shutil.move(source_path, destination_path)
    
# data_M의 이미지를 data 폴더로 이동
for filename in os.listdir(data_path_M):
    source_path = os.path.join(data_path_M, filename)
    destination_path = os.path.join(merged_folder, filename)
    shutil.move(source_path, destination_path)

# 빈 폴더 제거
os.rmdir(data_path_F)
os.rmdir(data_path_M)

In [9]:
# 남여 데이터프레임 합치고 필요한 열만 가져옴
df = pd.concat([df_f, df_m], axis = 0).reset_index(drop = True)
df['No'] = df.index + 1
df['No'] = df['No'].apply(lambda x : str(x) + '.jpg')

df['boneage'] = (df['BA 1'] + df['BA 2']) / 2
df['성별'] = df['성별'].map({2 : 'F', 1:'M'})

In [10]:
df.to_csv('골밀도 데이터/total_data.csv', index = 0)